In [ ]:
%matplotlib inline
import re
import os

import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset

os.environ['PSRAD_PATH'] = '/scratch/uni/u237/users/lkluft/icon-aes/psrad/'
import conrad
import typhon


plt.style.use(typhon.plots.styles('typhon'))

def extract_scale(filename, regex='.*co2_x(.*)-.*\.nc'):
    """Extract the CO2 increase from filename.
    
    Parameters:
        filename (str): Path to netCDF file.
        
    Returns:
        str: Extracted CO2 factor if successful, filename if not.
        
    Examples:
        >>> extract_scale('results/tropical_co2_x0.5-conv_fVMR.nc')
        '0.5'
    """
    pattern = re.compile(regex)
    matches = pattern.match(filename)

    if matches:
        return matches.group(1)
    else:
        return filename
    

def get_legend_label(filename, label=r'$CO_2 \times {placeholder}$', **kwargs):
    """Get label for legend entry."""
    experiment = extract_scale(filename, **kwargs)
    
    return label.format(placeholder=experiment)
    

# Refined grid visualisation

In [ ]:
start, stop, num = 1013e2, 0.01e2, 15

p_refined = conrad.utils.refined_pgrid(start, stop, num)
p_log = typhon.math.nlogspace(start, stop, num)
p_lin = np.linspace(start, stop, num)

fig, ax = plt.subplots(figsize=(4, 8))
ax.axhline(100e2, color='darkred', linestyle='dashed')
typhon.plots.profile_p_log(p_lin, 0* np.ones(p_lin.size), marker='o', linestyle='none')
typhon.plots.profile_p_log(p_log, 1 * np.ones(p_log.size), marker='o', linestyle='none')
typhon.plots.profile_p_log(p_refined, 2 * np.ones(p_refined.size), marker='o', linestyle='none')
ax.set_xticks(range(3))
ax.set_xticklabels(['Linear', 'Logarithmic', 'Refined'], rotation=0)
ax.grid(True)
fig.savefig('plots/refined_pgrid.pdf')

# Refined grid results

In [ ]:
season = 'tropical'

grid_log = sorted(
    glob.glob(f'results/{season}_layers_*.nc'),
    key=lambda fname: float(conrad.utils.extract_metadata(fname).scale),
    reverse=True,
)

grid_refined = sorted(
    glob.glob(f'results/{season}_refined-layers_*.nc'),
    key=lambda fname: float(conrad.utils.extract_metadata(fname).scale),
    reverse=True,
)

if len(grid_log) * len(grid_refined) == 0:
    raise Exception('No input files found. Probably screwed glob pattern.')

legend_entry = ['Logarithmic', 'Refined']

fig, ax = plt.subplots()
for n, grid_type in enumerate([grid_log, grid_refined]):
    number_of_levels = [
        [int(conrad.utils.extract_metadata(f).scale), float(Dataset(f).variables['temperature'][-1])]
        for f in grid_type
    ]
    nlevels, Ts_eq = np.split(np.array(number_of_levels), 2, axis=1)

    ax.plot(nlevels, Ts_eq, marker='o', linestyle='none',
            color=f'C{n}', label=legend_entry[n])
    ax.plot(nlevels, Ts_eq, linestyle='--', alpha=0.5, color=f'C{n}')
ax.grid(True)
ax.set_xlabel('Pressure levels [#]')
ax.set_ylabel('$T_{s, eq}$ [K]')
ax.set_ylim(309.3, 313.5)
ax.legend()
fig.savefig('plots/layer_dependency_refined.pdf')

# Improved algorithm

In [ ]:
dataset_reference = Dataset('results/tropical_refined-layers_1000.nc')
dataset_improved = Dataset('results/tropical_refined-layers_1000_fast.nc')

days = dataset_reference.variables['time'][:] / 24
T_reference = dataset_reference.variables['T'][:, 100]
T_improved = dataset_improved.variables['T'][:, 100]

fig, ax = plt.subplots()
ax.plot(days, (T_improved - T_reference) / T_reference)
ax.set_xlabel('Days')
ax.set_xlim(left=0)
ax.set_ylabel('Relative difference')
ax.grid(True)

fig.savefig('plots/improved_scheme_error.pdf')